# ML Lab 04 "solutions"

Regression.

## Imports

In [1]:
# Data and Datasets
import pandas as pd
from sklearn.datasets import fetch_california_housing

# Data processing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer, quantile_transform

# Validation methods
from sklearn.model_selection import train_test_split

# Metrics package
from sklearn import metrics

# Regression metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error

# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
from sklearn.tree import export_graphviz

# Utils
from sklearn.pipeline import Pipeline
import pprint
import numpy as np
from time import time

## Utility functions

### Evaluation Function

Introduced this function, to save on repetitive code. In summary, what this function does:

* Takes as input arguments: a dataset (X and y), name of a model, a model object, and number of times to run hold-out evaluation
* It then does hold-out validation, for each run, training and testing the model
* It finally returns three arrays with the performance metrics from each hold-out validation run
  - Mean Squared Error
  - Mean Absolute Error
  - Maximum Error

In [2]:
def validate_model(X, y, model, num_runs=1000, test_size=0.2, output=True):
    """
    Performs hold-out validation of a given model on the dataset provided (X, y).
    The default number of runs is 1000, and the default training/test split is 80/20.
    """
    
    # arrays for storing performance metrics
    mse_list = np.array([])
    mae_list = np.array([])
    me_list = np.array([])

    # capturing the start-time
    start_time = time()

    if output:
        print("Starting model validation")
    
    # runing `num_runs` number of validation runs of data splitting, training and testing
    for x in range(num_runs):
        # Hold-out validation - default 80% training and 20% testing
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None)

        # Training and testing the model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Appending performance metrics to arrays created above
        mse_list = np.append(mse_list, mean_squared_error(y_test, y_pred))
        mae_list = np.append(mae_list, mean_absolute_error(y_test, y_pred))
        me_list = np.append(me_list, max_error(y_test, y_pred))

    # capturing the end-time and printing out
    end_time = time()
    
    if output:
        print("Completed %d validation runs in %.2f seconds\n" % (num_runs, end_time - start_time))
    
    return mse_list, mae_list, me_list

### Result function

To save and present results in a nice manner, the following code sets up a DataFrame for results and a function to add data to it, which we'll use further below when we're testing the different regression models. 

- Metrics: https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics

In [3]:
result_df = pd.DataFrame(columns=['Model','MSE','MAE','ME'])

def add_result(model_name, mse_list, mae_list, me_list):
    """
    Add a result row to a pandas dataframe (created above) for a model with name (model_name).
    mse_list, mae_list and me_list are array lists of performance results from different hold-out validation runs,
    coming from the validate_model() function, above.
    """
    global result_df # doing this to be able to access the gloval dataframe defined above the function
    
    new_row = {'Model':model_name, 'MSE':mse_list.mean(), 'MAE':mae_list.mean(), 'ME':me_list.mean()}
    result_df = pd.concat([result_df, pd.Series(new_row).to_frame().T])
    
    return result_df

def print_results(mse_list, mae_list, me_list):
    """
    Prints out the mean MSE, MAE and ME + 2 x standard deviation (STD).
    We multiply the STD by 2 because 95% of the data around the mean is within 2 standard deviations, 
    so it gives us a good idea of the range of values around the mean.
    """
    print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
    print("MAE:  %0.2f (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
    print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))


### Default variables

In [4]:
# Sets the number of runs of hold-out validation below (from which we get mean and stdev statistics)
# PS: setting this here, so that we have just a single place to change this, instead of in 10+ function calls further below
num_runs = 100

## Loading dataset

In [5]:
# The following is required if you run this on a Mac as you'll otherwise get an error trying to fetch the dataset in the code call below
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

# Loading the dataset
dataset = fetch_california_housing()

print(dataset['DESCR'])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [6]:
# Convert to Pandas DataFrame (only taking the feature values 1st)
california_df = pd.DataFrame(dataset.data,columns=dataset.feature_names)

# The target variable is the median house value for California districts,
# expressed in hundreds of thousands of dollars ($100,000)
california_df['target'] = dataset.target

# Storing references to feature names
features = dataset['feature_names']

# Get the X (feature matrix) and y (target vector) from the data
X, y = dataset.data, dataset.target

# And just to ensure we've loaded what we expect..
california_df.head()

MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  target  
0    -122.23   4.526  
1    -122.22   3.585  
2    -122.24   3.521  
3    -122.25   3.413  
4    -122.25   3.422

In [7]:
california_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  float64
 5   AveOccup    20640 non-null  float64
 6   Latitude    20640 non-null  float64
 7   Longitude   20640 non-null  float64
 8   target      20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


## Testing Linear Regression

This will do a single run of hold-out validation for a Linear Regression model, printing out the MSE, MAE and ME.

In [8]:
# Split the dataset into training/test portions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Instantiate linear regression model
lireg_model = LinearRegression()

# Train the model on the training data
lireg_model.fit(X_train, y_train)

# Test the model on the testing data
y_pred = lireg_model.predict(X_test)

# Calculating performance metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
me = max_error(y_test, y_pred)

# Printing performance metrics to screen
print("MSE:" , mse)
print("MAE:" , mae)
print("ME: " , me)

MSE: 0.52480817907834
MAE: 0.5364076376149008
ME:  4.023346443555237


## Linear Regression vs Polynomial Regression vs CART

The following code uses the utility functions defined above, running multiple runs of hold-out validation and storing the results to a Pandas DataFrame that you can print out in a neat table.

API references: 

- Linear Regression: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
- Polynomial Regression: https://scikit-learn.org/stable/modules/linear_model.html#polynomial-regression-extending-linear-models-with-basis-functions
- CART: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

### Linear Regression

In [9]:
# Instantiate model
lireg_model = LinearRegression()

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, lireg_model, num_runs)

# save the result
add_result('Linear Regression', mse_list, mae_list, me_list)

# printing out the performance metrics
print_results(mse_list, mae_list, me_list)

Starting model validation
Completed 100 validation runs in 1.01 seconds

MSE:  0.55 (+/- 0.29)
MAE:  0.53 (+/- 0.01)
ME:   7.12 (+/- 15.44)


### Polynomial Regression

Doing 1st, 2nd and 3rd order polynomials here, in each respective cell below.

Btw, here's an image of how different polynomials look like.

!["Polynomial regression functions"](https://scontent-lhr8-2.xx.fbcdn.net/v/t1.6435-9/84718027_166245788135822_4605281620893630464_n.png?_nc_cat=105&ccb=1-7&_nc_sid=7f8c78&_nc_ohc=vqUSrRVbJE8AX-KINyT&_nc_ht=scontent-lhr8-2.xx&oh=00_AfAFEgZyNrytSsA4_6h7MSx_TKhtBKlpm4QzgRIskxmScQ&oe=6555FE37)


In [10]:
# Instantiate model
poly_model = Pipeline([('poly', PolynomialFeatures(degree=1)),
                       ('linear', LinearRegression())])

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model, num_runs)

# save the result
add_result('Polynomial Regression (1st Order)', mse_list, mae_list, me_list)

# printing out the performance metrics
print_results(mse_list, mae_list, me_list)

Starting model validation
Completed 100 validation runs in 1.05 seconds

MSE:  0.54 (+/- 0.16)
MAE:  0.53 (+/- 0.01)
ME:   5.90 (+/- 9.51)


In [11]:
# Instantiate model
poly_model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                       ('linear', LinearRegression())])

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model, num_runs)

# save the result
add_result('Polynomial Regression (2nd Order)', mse_list, mae_list, me_list)

# printing out the performance metrics
print_results(mse_list, mae_list, me_list)

Starting model validation
Completed 100 validation runs in 31.73 seconds

MSE:  15.80 (+/- 104.10)
MAE:  0.49 (+/- 0.14)
ME:   98.31 (+/- 429.17)


In [12]:
# Instantiate model
poly_model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                       ('linear', LinearRegression())])

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model, num_runs)

# save the result
add_result('Polynomial Regression (3rd Order)', mse_list, mae_list, me_list)

# printing out the performance metrics
print_results(mse_list, mae_list, me_list)

Starting model validation
Completed 100 validation runs in 37.48 seconds

MSE:  882120.56 (+/- 7652428.25)
MAE:  6.23 (+/- 31.53)
ME:   21160.46 (+/- 111486.48)


### CART

In [13]:
# Instantiate model
cart_model = DecisionTreeRegressor()

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, cart_model, num_runs)

# save the result
add_result('CART', mse_list, mae_list, me_list)

# printing out the performance metrics
print_results(mse_list, mae_list, me_list)

Starting model validation
Completed 100 validation runs in 12.60 seconds

MSE:  0.52 (+/- 0.04)
MAE:  0.46 (+/- 0.02)
ME:   4.34 (+/- 0.49)


### Results

Which model appears to be best? and, is it best according to all metrics?

In [14]:
# Printing out the results table, ordered by MSE
result_df.sort_values(by=['MSE'])

Model            MSE       MAE            ME
0                               CART       0.519687  0.460709      4.344249
0  Polynomial Regression (1st Order)       0.535894  0.531902      5.896324
0                  Linear Regression       0.552139  0.531209      7.118273
0  Polynomial Regression (2nd Order)      15.802336  0.493902     98.314153
0  Polynomial Regression (3rd Order)  882120.562825   6.23457  21160.458974

## Data transformation of target

Using a quantile transformer here, based on [[this](https://scikit-learn.org/stable/auto_examples/compose/plot_transformed_target.html#sphx-glr-auto-examples-compose-plot-transformed-target-py)] article from Scikit-Learn.

### Training and testing the models

In [15]:
# Instantiate model
lireg_model_trans = TransformedTargetRegressor(
    regressor=LinearRegression(),
    transformer=QuantileTransformer(n_quantiles=200,
                                    output_distribution='normal'))

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, lireg_model_trans, num_runs)

# save the result
add_result('Linear Regression (trans)', mse_list, mae_list, me_list)

# printing out the performance metrics
print_results(mse_list, mae_list, me_list)

Starting model validation
Completed 100 validation runs in 1.43 seconds

MSE:  0.58 (+/- 0.04)
MAE:  0.54 (+/- 0.02)
ME:   4.36 (+/- 0.49)


In [16]:
# Instantiate model
poly_model_trans = TransformedTargetRegressor(
    regressor=Pipeline([('poly', PolynomialFeatures(degree=1)),
                       ('linear', LinearRegression())]),
    transformer=QuantileTransformer(n_quantiles=200,
                                    output_distribution='normal'))

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model_trans, num_runs)

# save the result
add_result('Polynomial Regression (1st Order) (trans)', mse_list, mae_list, me_list)

# printing out the performance metrics
print_results(mse_list, mae_list, me_list)

Starting model validation
Completed 100 validation runs in 1.48 seconds

MSE:  0.58 (+/- 0.04)
MAE:  0.54 (+/- 0.02)
ME:   4.28 (+/- 0.54)


In [17]:
# Instantiate model
poly_model_trans = TransformedTargetRegressor(
    regressor=Pipeline([('poly', PolynomialFeatures(degree=2)),
                       ('linear', LinearRegression())]),
    transformer=QuantileTransformer(n_quantiles=200,
                                    output_distribution='normal'))

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model_trans, num_runs)

# save the result
add_result('Polynomial Regression (2nd Order) (trans)', mse_list, mae_list, me_list)

# printing out the performance metrics
print_results(mse_list, mae_list, me_list)

Starting model validation
Completed 100 validation runs in 30.73 seconds

MSE:  0.50 (+/- 0.04)
MAE:  0.48 (+/- 0.02)
ME:   4.53 (+/- 0.72)


In [18]:
# Instantiate model
poly_model_trans = TransformedTargetRegressor(
    regressor=Pipeline([('poly', PolynomialFeatures(degree=3)),
                       ('linear', LinearRegression())]),
    transformer=QuantileTransformer(n_quantiles=200,
                                    output_distribution='normal'))

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model_trans, num_runs)

# save the result
add_result('Polynomial Regression (3rd Order) (trans)', mse_list, mae_list, me_list)

# printing out the performance metrics
print_results(mse_list, mae_list, me_list)

Starting model validation
Completed 100 validation runs in 36.69 seconds

MSE:  0.48 (+/- 0.04)
MAE:  0.46 (+/- 0.02)
ME:   4.37 (+/- 0.66)


In [19]:
# Instantiate model
cart_model_trans = TransformedTargetRegressor(
    regressor=DecisionTreeRegressor(),
    transformer=QuantileTransformer(n_quantiles=200,
                                    output_distribution='normal'))

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, cart_model_trans, num_runs)

# save the result
add_result('CART (trans)', mse_list, mae_list, me_list)

# printing out the performance metrics
print_results(mse_list, mae_list, me_list)

Starting model validation
Completed 100 validation runs in 13.57 seconds

MSE:  0.51 (+/- 0.05)
MAE:  0.45 (+/- 0.02)
ME:   4.32 (+/- 0.37)


### Results

Did the performance of any of the models improve when transforming the target values?

In [20]:
# Printing out the results table, ordered by MSE
result_df.sort_values(by=['MSE'])

Model            MSE       MAE  \
0  Polynomial Regression (3rd Order) (trans)       0.481761  0.463337   
0  Polynomial Regression (2nd Order) (trans)       0.504109   0.48315   
0                               CART (trans)        0.50613  0.446679   
0                                       CART       0.519687  0.460709   
0          Polynomial Regression (1st Order)       0.535894  0.531902   
0                          Linear Regression       0.552139  0.531209   
0                  Linear Regression (trans)       0.576177  0.536117   
0  Polynomial Regression (1st Order) (trans)       0.579006  0.536407   
0          Polynomial Regression (2nd Order)      15.802336  0.493902   
0          Polynomial Regression (3rd Order)  882120.562825   6.23457   

             ME  
0      4.367338  
0      4.526621  
0      4.315338  
0      4.344249  
0      5.896324  
0      7.118273  
0      4.355939  
0      4.278824  
0     98.314153  
0  21160.458974